In [ ]:
# Copyright 2020 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

## Overview

In the previous notebook we explained and showed how we can preprocess data with multi-hot columns with NVTabular, and train an TF MLP model using NVTabular KerasSequenceLoader. We learned how to save a workflow, a trained TF model, and the ensemble model. In this notebook, we will show example request scripts sent to triton inference server

- to transform new/streaming data with NVTabular library
- to deploy the end-to-end pipeline to generate prediction results for new data from trained TF model

## Getting Started

In [ ]:
# External dependencies
import os
from os import path                
import time
import gc

import nvtabular
import cudf 
from tritonclient.utils import *
import tritonclient.grpc as grpcclient
import nvtabular.inference.triton as nvt_triton

We define our base directory, containing the data.

In [ ]:
# path to store raw and preprocesses data
MODEL_BASE_DIR = os.environ.get('MODEL_BASE_DIR', '/model/models/')
INPUT_DATA_DIR = os.environ.get('INPUT_DATA_DIR', '/model/data/')

## Verify Triton Is Running Correctly

In [ ]:
!apt-get install curl -y

In [ ]:
!curl -i localhost:8000/v2/health/ready

## Send request to Triton IS to transform raw dataset

Let's read the raw validation set, and send 3 rows of `userid` and `movieId` as input to the saved NVTabular model.

In [ ]:
df_valid = cudf.read_parquet(INPUT_DATA_DIR + 'valid.parquet')
df_valid.head()

In [ ]:
df = cudf.DataFrame({"userId": [99476, 107979, 155372], "movieId": [104374, 2634, 1614]})
inputs = nvt_triton.convert_df_to_triton_input(["userId", "movieId"], df, grpcclient.InferInput)

outputs = [
    grpcclient.InferRequestedOutput(col)
    for col in ["userId", "movieId", "genres__nnzs", "genres__values"]
]

MODEL_NAME_NVT = os.environ.get('MODEL_NAME_NVT', 'movielens_mh_nvt')

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer(MODEL_NAME_NVT, inputs, request_id="1", outputs=outputs)

for col in ["userId", "movieId", "genres__nnzs", "genres__values"]:
    print(col, response.as_numpy(col), response.as_numpy(col).shape)

You might notice that we don't need to send the genres column as an input. The reason for that is the nvt model will look up the genres for each movie as part of the `JoinExternal` op it applies. Also notice that when creating the request for the `movielens_mh_nvt` model, we return 2 columns (values and nnzs) for the `genres` column rather than 1.

## END-2-END INFERENCE PIPELINE

We will do the same, but this time we directly read in first 3 rows of the the raw `valid.parquet` file with cuDF.

In [ ]:
# read in the workflow (to get input/output schema to call triton with)
batch = cudf.read_parquet(os.path.join(INPUT_DATA_DIR, "valid.parquet"), num_rows=3, columns=['userId', 'movieId'])
print(batch, "\n")

# convert the batch to a triton inputs
inputs = nvt_triton.convert_df_to_triton_input(["userId", "movieId"], batch, grpcclient.InferInput)

# placeholder variables for the output
outputs = [grpcclient.InferRequestedOutput("dense_3")]

MODEL_NAME_ENSEMBLE = os.environ.get('MODEL_NAME_ENSEMBLE', 'movielens_mh')

# build a client to connect to our server. 
# This InferenceServerClient object is what we'll be using to talk to Triton.
# make the request with tritonclient.grpc.InferInput object

with grpcclient.InferenceServerClient("localhost:8001") as client:
    response = client.infer(MODEL_NAME_ENSEMBLE, inputs, request_id="1",outputs=outputs)

print("predicted softmax result:\n", response.as_numpy('dense_3'))


In [ ]:
del df_valid
gc.collect()